# DMA AXI Stream FIFO Test for PYNQ

# DMA Master connected to S_AXI_HP2 Port of Zynq System via AXI Interconnect (axi_mem_intercon) used by tracebufer_pmods/audrino
#DMA S_AXI_LITE port connected to AXI Interconnect (processing_system7_0_axi_periph)
#System Block diagram at: https://github.com/IamVNIE/hapi-des/blob/master/notebook/axi-stream-fifo-test/pynq.xps
#DMA with FIFO created using: http://www.fpgadeveloper.com/2014/08/using-the-axi-dma-in-vivado.html
#Thanks to Peter Ogden at Xilinx for the solution for DMA buffer accesses from Python

In [1]:
from pynq import Overlay
ol = Overlay("axi_stream.bit")

ol.download()
ol.bitstream.timestamp

from pynq import PL
PL.bitfile_name

ol.is_loaded()

True

# Testing dma class provided in Base PYNQ  

In [2]:
from pynq.drivers.dma import DMA
dma_base = int(PL.ip_dict["SEG_axi_dma_0_Reg2"][0],16)
print(hex(dma_base))
dma = DMA(dma_base, direction=3)

0x40400000


In [10]:
import numpy as np
from random import randint
import time
data2Transmit=[randint(0,x) for x in range(1000000) ]
size=int(len(data2Transmit)/2*8)
dma.create_buf(size)
from pynq.drivers.dma import ffi
buf = ffi.buffer(dma.buf, size)
view = np.frombuffer(buf, np.int32,-1)
#print(view)
view[:] = data2Transmit
#Uncomment the following line to see what input values are transferred to DMA
#print(view)
tx_strt = time.time()
result = view.copy()
tx_done=time.time()
rx_strt = time.time()
vin = dma.get_buf()
rx_done=time.time()
## ----For Statistics --- ##
rx= rx_done - rx_strt
tx= tx_done - tx_strt
txLen=len(data2Transmit)*32
txbps=len(data2Transmit)*32/tx
rxbps=len(data2Transmit)*32/rx
txMbps=(len(data2Transmit)*32/tx)/1e6
rxMbps=(len(data2Transmit)*32/rx)/1e6
txGbps=(len(data2Transmit)*32/tx)/1e9
rxGbps=(len(data2Transmit)*32/rx)/1e9
print("TOTAL TIME TO SEND - ",txLen, "BITS = ", tx)
print("TOTAL TIME TO RECV - ",txLen, "BITS = ", rx)
print("TX BW = ",txbps, "bps | ",txMbps,"Mbps | ",txGbps,"Gbps")
print("RX BW = ",rxbps, "bps | ",rxMbps,"Mbps | ",rxGbps,"Gbps")
print("TOTAL BW = ",txMbps+rxMbps,"Mbps | ",txGbps+rxGbps,"Gbps")

dataRecieved=[]

for i in range(len(view)):
    dataRecieved.append(vin[i])
#Uncomment the following line to see what output values are transferred from DMA 
#print(dataRecieved)


TOTAL TIME TO SEND -  32000000 BITS =  0.06726622581481934
TOTAL TIME TO RECV -  32000000 BITS =  0.0007665157318115234
TX BW =  475721650.98268557 bps |  475.72165098268556 Mbps |  0.47572165098268554 Gbps
RX BW =  41747349300.15552 bps |  41747.34930015552 Mbps |  41.74734930015552 Gbps
TOTAL BW =  42223.0709511382 Mbps |  42.223070951138205 Gbps
